In [1]:
from value_today.scraper import Scraper

In [2]:
scraper = Scraper()

In [3]:
scraper.number_companies()

5083

In [4]:
raw_data = scraper.valuetoday_usa_scraper(20)

In [5]:
valuetoday_df = scraper.df_cleaning(raw_data)

In [7]:
scraper.save_csv(valuetoday_df, "final_data_example")